### CEEMDAN-SSA-VMD--WOA-CNN-ResNet-BiGRU-Attention（The hyperparameters have been optimized.）

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import train_test_split
import os
from PyEMD import CEEMDAN  # 确保你已经安装了PyEMD库
from vmdpy import VMD  # 确保你已经安装了vmdpy库

# 固定随机种子
seed = 42  # 可以选择任意整数作为种子
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 读取数据
data = pd.read_csv('D:/Jupyter/Data/Source code data/单一特征/龙门径流.csv')['flow'].values
# CCEMDAN分解
ceemdan = CEEMDAN()
IMFs = ceemdan(data)
K_ceemdan = len(IMFs)  # CCEMDAN模态数

# 对第一个和第二个模态进行VMD分解
alpha = 1565  # 带宽限制
tau = 0  # 时间平滑参数
K_vmd = 7  # VMD分解的模态数
DC = 0  # 直流成分
init = 1  # 初始化方法
tol = 1e-7  # 收敛容差

u1, u_hat1, omega1 = VMD(IMFs[0], alpha, tau, K_vmd, DC, init, tol)
u2, u_hat2, omega2 = VMD(IMFs[1], alpha, tau, K_vmd, DC, init, tol)

# 数据预处理
scalers_ceemdan = [MinMaxScaler() for _ in range(K_ceemdan)]
data_scaled_ceemdan = [scalers_ceemdan[i].fit_transform(IMFs[i].reshape(-1, 1)) for i in range(K_ceemdan)]

scalers_vmd1 = [MinMaxScaler() for _ in range(K_vmd)]
data_scaled_vmd1 = [scalers_vmd1[i].fit_transform(u1[i].reshape(-1, 1)) for i in range(K_vmd)]

scalers_vmd2 = [MinMaxScaler() for _ in range(K_vmd)]
data_scaled_vmd2 = [scalers_vmd2[i].fit_transform(u2[i].reshape(-1, 1)) for i in range(K_vmd)]

# 创建时间窗口数据集
def create_dataset(dataset, look_back=12):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

look_back = 12
Xs_ceemdan, ys_ceemdan = [], []
for i in range(K_ceemdan):
    X, y = create_dataset(data_scaled_ceemdan[i], look_back)
    Xs_ceemdan.append(X)
    ys_ceemdan.append(y)

Xs_vmd1, ys_vmd1 = [], []
for i in range(K_vmd):
    X, y = create_dataset(data_scaled_vmd1[i], look_back)
    Xs_vmd1.append(X)
    ys_vmd1.append(y)

Xs_vmd2, ys_vmd2 = [], []
for i in range(K_vmd):
    X, y = create_dataset(data_scaled_vmd2[i], look_back)
    Xs_vmd2.append(X)
    ys_vmd2.append(y)

# 划分数据集
# 划分数据集
X_trains_ceemdan, X_vals_ceemdan, X_tests_ceemdan, y_trains_ceemdan, y_vals_ceemdan, y_tests_ceemdan = [], [], [], [], [], []
for i in range(K_ceemdan):
    data_length = len(Xs_ceemdan[i])
    train_end = int(data_length * 0.7)
    val_end = int(data_length * 0.85)
    X_train = Xs_ceemdan[i][:train_end]
    X_val = Xs_ceemdan[i][train_end:val_end]
    X_test = Xs_ceemdan[i][val_end:]
    y_train = ys_ceemdan[i][:train_end]
    y_val = ys_ceemdan[i][train_end:val_end]
    y_test = ys_ceemdan[i][val_end:]
    X_trains_ceemdan.append(X_train)
    X_vals_ceemdan.append(X_val)
    X_tests_ceemdan.append(X_test)
    y_trains_ceemdan.append(y_train)
    y_vals_ceemdan.append(y_val)
    y_tests_ceemdan.append(y_test)

X_trains_vmd1, X_vals_vmd1, X_tests_vmd1, y_trains_vmd1, y_vals_vmd1, y_tests_vmd1 = [], [], [], [], [], []
for i in range(K_vmd):
    data_length = len(Xs_vmd1[i])
    train_end = int(data_length * 0.7)
    val_end = int(data_length * 0.85)
    X_train = Xs_vmd1[i][:train_end]
    X_val = Xs_vmd1[i][train_end:val_end]
    X_test = Xs_vmd1[i][val_end:]
    y_train = ys_vmd1[i][:train_end]
    y_val = ys_vmd1[i][train_end:val_end]
    y_test = ys_vmd1[i][val_end:]
    X_trains_vmd1.append(X_train)
    X_vals_vmd1.append(X_val)
    X_tests_vmd1.append(X_test)
    y_trains_vmd1.append(y_train)
    y_vals_vmd1.append(y_val)
    y_tests_vmd1.append(y_test)

X_trains_vmd2, X_vals_vmd2, X_tests_vmd2, y_trains_vmd2, y_vals_vmd2, y_tests_vmd2 = [], [], [], [], [], []
for i in range(K_vmd):
    data_length = len(Xs_vmd2[i])
    train_end = int(data_length * 0.7)
    val_end = int(data_length * 0.85)
    X_train = Xs_vmd2[i][:train_end]
    X_val = Xs_vmd2[i][train_end:val_end]
    X_test = Xs_vmd2[i][val_end:]
    y_train = ys_vmd2[i][:train_end]
    y_val = ys_vmd2[i][train_end:val_end]
    y_test = ys_vmd2[i][val_end:]
    X_trains_vmd2.append(X_train)
    X_vals_vmd2.append(X_val)
    X_tests_vmd2.append(X_test)
    y_trains_vmd2.append(y_train)
    y_vals_vmd2.append(y_val)
    y_tests_vmd2.append(y_test)
# 转换为Tensor
X_trains_ceemdan = [torch.tensor(X_train, dtype=torch.float32).unsqueeze(2).to(device) for X_train in X_trains_ceemdan]
y_trains_ceemdan = [torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device) for y_train in y_trains_ceemdan]
X_vals_ceemdan = [torch.tensor(X_val, dtype=torch.float32).unsqueeze(2).to(device) for X_val in X_vals_ceemdan]
y_vals_ceemdan = [torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(device) for y_val in y_vals_ceemdan]
X_tests_ceemdan = [torch.tensor(X_test, dtype=torch.float32).unsqueeze(2).to(device) for X_test in X_tests_ceemdan]
y_tests_ceemdan = [torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device) for y_test in y_tests_ceemdan]

X_trains_vmd1 = [torch.tensor(X_train, dtype=torch.float32).unsqueeze(2).to(device) for X_train in X_trains_vmd1]
y_trains_vmd1 = [torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device) for y_train in y_trains_vmd1]
X_vals_vmd1 = [torch.tensor(X_val, dtype=torch.float32).unsqueeze(2).to(device) for X_val in X_vals_vmd1]
y_vals_vmd1 = [torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(device) for y_val in y_vals_vmd1]
X_tests_vmd1 = [torch.tensor(X_test, dtype=torch.float32).unsqueeze(2).to(device) for X_test in X_tests_vmd1]
y_tests_vmd1 = [torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device) for y_test in y_tests_vmd1]

X_trains_vmd2 = [torch.tensor(X_train, dtype=torch.float32).unsqueeze(2).to(device) for X_train in X_trains_vmd2]
y_trains_vmd2 = [torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device) for y_train in y_trains_vmd2]
X_vals_vmd2 = [torch.tensor(X_val, dtype=torch.float32).unsqueeze(2).to(device) for X_val in X_vals_vmd2]
y_vals_vmd2 = [torch.tensor(y_val, dtype=torch.float32).unsqueeze(1).to(device) for y_val in y_vals_vmd2]
X_tests_vmd2 = [torch.tensor(X_test, dtype=torch.float32).unsqueeze(2).to(device) for X_test in X_tests_vmd2]
y_tests_vmd2 = [torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device) for y_test in y_tests_vmd2]

# 创建DataLoader
train_loaders_ceemdan = [DataLoader(TensorDataset(X_trains_ceemdan[i], y_trains_ceemdan[i]), batch_size=32, shuffle=True) for i in range(K_ceemdan)]
val_loaders_ceemdan = [DataLoader(TensorDataset(X_vals_ceemdan[i], y_vals_ceemdan[i]), batch_size=32, shuffle=False) for i in range(K_ceemdan)]
test_loaders_ceemdan = [DataLoader(TensorDataset(X_tests_ceemdan[i], y_tests_ceemdan[i]), batch_size=32, shuffle=False) for i in range(K_ceemdan)]

train_loaders_vmd1 = [DataLoader(TensorDataset(X_trains_vmd1[i], y_trains_vmd1[i]), batch_size=32, shuffle=True) for i in range(K_vmd)]
val_loaders_vmd1 = [DataLoader(TensorDataset(X_vals_vmd1[i], y_vals_vmd1[i]), batch_size=32, shuffle=False) for i in range(K_vmd)]
test_loaders_vmd1 = [DataLoader(TensorDataset(X_tests_vmd1[i], y_tests_vmd1[i]), batch_size=32, shuffle=False) for i in range(K_vmd)]

train_loaders_vmd2 = [DataLoader(TensorDataset(X_trains_vmd2[i], y_trains_vmd2[i]), batch_size=32, shuffle=True) for i in range(K_vmd)]
val_loaders_vmd2 = [DataLoader(TensorDataset(X_vals_vmd2[i], y_vals_vmd2[i]), batch_size=32, shuffle=False) for i in range(K_vmd)]
test_loaders_vmd2 = [DataLoader(TensorDataset(X_tests_vmd2[i], y_tests_vmd2[i]), batch_size=32, shuffle=False) for i in range(K_vmd)]

# 定义多头注意力模块
class MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.hidden_dim = hidden_dim
        self.Wq = nn.Linear(input_dim, hidden_dim * num_heads)
        self.Wk = nn.Linear(input_dim, hidden_dim * num_heads)
        self.Wv = nn.Linear(input_dim, hidden_dim * num_heads)
        self.fc = nn.Linear(hidden_dim * num_heads, input_dim)

    def forward(self, x):
        batch_size, seq_length, _ = x.size()
        
        # 生成 Q, K, V
        Q = self.Wq(x).view(batch_size, seq_length, self.num_heads, self.hidden_dim).transpose(1, 2)  # (batch_size, num_heads, seq_length, hidden_dim)
        K = self.Wk(x).view(batch_size, seq_length, self.num_heads, self.hidden_dim).transpose(1, 2)
        V = self.Wv(x).view(batch_size, seq_length, self.num_heads, self.hidden_dim).transpose(1, 2)
        
        # 计算注意力分数
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.hidden_dim ** 0.5)  # (batch_size, num_heads, seq_length, seq_length)
        attention_weights = torch.softmax(attention_scores, dim=-1)  # (batch_size, num_heads, seq_length, seq_length)
        
        # 加权平均
        context = torch.matmul(attention_weights, V)  # (batch_size, num_heads, seq_length, hidden_dim)
        context = context.transpose(1, 2).contiguous().view(batch_size, seq_length, -1)  # (batch_size, seq_length, hidden_dim * num_heads)
        
        output = self.fc(context)  # (batch_size, seq_length, input_dim)
        return output

# 定义CNN-Resnet-BiGRU-MultiHeadAttention模型
class CNN_Resnet_BiGRU_MultiHeadAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, num_heads):
        super(CNN_Resnet_BiGRU_MultiHeadAttention, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # CNN部分
        self.conv1 = nn.Conv1d(input_dim, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=3, padding=1)
        
        # BiGRU部分
        self.bigru = nn.GRU(256, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        
        # 多头注意力部分
        self.multihead_attention = MultiHeadAttention(hidden_dim * 2, hidden_dim, num_heads)
        
        # 全连接层
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        # 调整张量的维度顺序
        x = x.permute(0, 2, 1)  # 从 [batch_size, sequence_length, input_dim] 变为 [batch_size, input_dim, sequence_length]
        
        # CNN部分
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool(x)
        
        # 调整形状以适应BiGRU输入
        x = x.permute(0, 2, 1)
        
        # BiGRU部分
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_dim).to(device)
        out, _ = self.bigru(x, h0)
        
        # 多头注意力部分
        out = self.multihead_attention(out)
        
        # 选择最后一个时间步的输出
        out = out[:, -1, :]
        
        # 全连接层
        out = self.dropout(out)
        out = self.fc(out)
        return out

input_dim = 1
hidden_dim = 64
num_layers = 2
output_dim = 1
num_heads = 4  # 设置多头注意力的头数
models_ceemdan = [CNN_Resnet_BiGRU_MultiHeadAttention(input_dim, hidden_dim, num_layers, output_dim, num_heads).to(device) for _ in range(K_ceemdan)]
models_vmd1 = [CNN_Resnet_BiGRU_MultiHeadAttention(input_dim, hidden_dim, num_layers, output_dim, num_heads).to(device) for _ in range(K_vmd)]
models_vmd2 = [CNN_Resnet_BiGRU_MultiHeadAttention(input_dim, hidden_dim, num_layers, output_dim, num_heads).to(device) for _ in range(K_vmd)]

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizers_ceemdan = [optim.Adam(models_ceemdan[i].parameters(), lr=0.0006) for i in range(K_ceemdan)]
optimizers_vmd1 = [optim.Adam(models_vmd1[i].parameters(), lr=0.0006) for i in range(K_vmd)]
optimizers_vmd2 = [optim.Adam(models_vmd2[i].parameters(), lr=0.0006) for i in range(K_vmd)]

# 训练模型
num_epochs = 150
train_losses_ceemdan = [[] for _ in range(K_ceemdan)]
val_losses_ceemdan = [[] for _ in range(K_ceemdan)]
train_losses_vmd1 = [[] for _ in range(K_vmd)]
val_losses_vmd1 = [[] for _ in range(K_vmd)]
train_losses_vmd2 = [[] for _ in range(K_vmd)]
val_losses_vmd2 = [[] for _ in range(K_vmd)]

for i in range(K_ceemdan):
    for epoch in range(num_epochs):
        models_ceemdan[i].train()
        epoch_train_loss = 0.0
        for j, (inputs, labels) in enumerate(train_loaders_ceemdan[i]):
            optimizers_ceemdan[i].zero_grad()
            outputs = models_ceemdan[i](inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizers_ceemdan[i].step()
            epoch_train_loss += loss.item()
        train_losses_ceemdan[i].append(epoch_train_loss / len(train_loaders_ceemdan[i]))
        models_ceemdan[i].eval()
        epoch_val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loaders_ceemdan[i]:
                outputs = models_ceemdan[i](inputs)
                loss = criterion(outputs, labels)
                epoch_val_loss += loss.item()
            val_losses_ceemdan[i].append(epoch_val_loss / len(val_loaders_ceemdan[i]))

        if (epoch + 1) % 10 == 0:
            print(f'CCEMDAN IMF {i+1} - Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_losses_ceemdan[i][-1]:.4f}, Val Loss: {val_losses_ceemdan[i][-1]:.4f}')

for i in range(K_vmd):
    for epoch in range(num_epochs):
        models_vmd1[i].train()
        epoch_train_loss = 0.0
        for j, (inputs, labels) in enumerate(train_loaders_vmd1[i]):
            optimizers_vmd1[i].zero_grad()
            outputs = models_vmd1[i](inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizers_vmd1[i].step()
            epoch_train_loss += loss.item()
        train_losses_vmd1[i].append(epoch_train_loss / len(train_loaders_vmd1[i]))

        models_vmd1[i].eval()
        epoch_val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loaders_vmd1[i]:
                outputs = models_vmd1[i](inputs)
                loss = criterion(outputs, labels)
                epoch_val_loss += loss.item()
            val_losses_vmd1[i].append(epoch_val_loss / len(val_loaders_vmd1[i]))

        if (epoch + 1) % 10 == 0:
            print(f'VMD1 IMF {i+1} - Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_losses_vmd1[i][-1]:.4f}, Val Loss: {val_losses_vmd1[i][-1]:.4f}')

for i in range(K_vmd):
    for epoch in range(num_epochs):
        models_vmd2[i].train()
        epoch_train_loss = 0.0
        for j, (inputs, labels) in enumerate(train_loaders_vmd2[i]):
            optimizers_vmd2[i].zero_grad()
            outputs = models_vmd2[i](inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizers_vmd2[i].step()
            epoch_train_loss += loss.item()
        train_losses_vmd2[i].append(epoch_train_loss / len(train_loaders_vmd2[i]))

        models_vmd2[i].eval()
        epoch_val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loaders_vmd2[i]:
                outputs = models_vmd2[i](inputs)
                loss = criterion(outputs, labels)
                epoch_val_loss += loss.item()
            val_losses_vmd2[i].append(epoch_val_loss / len(val_loaders_vmd2[i]))

        if (epoch + 1) % 10 == 0:
            print(f'VMD2 IMF {i+1} - Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_losses_vmd2[i][-1]:.4f}, Val Loss: {val_losses_vmd2[i][-1]:.4f}')

# 测试模型
y_preds_ceemdan = []
for i in range(K_ceemdan):
    models_ceemdan[i].eval()
    y_pred = []
    with torch.no_grad():
        for inputs, _ in test_loaders_ceemdan[i]:
            outputs = models_ceemdan[i](inputs)
            y_pred.extend(outputs.cpu().numpy())
    y_pred = np.array(y_pred)
    y_preds_ceemdan.append(y_pred)

y_preds_vmd1 = []
for i in range(K_vmd):
    models_vmd1[i].eval()
    y_pred = []
    with torch.no_grad():
        for inputs, _ in test_loaders_vmd1[i]:
            outputs = models_vmd1[i](inputs)
            y_pred.extend(outputs.cpu().numpy())
    y_pred = np.array(y_pred)
    y_preds_vmd1.append(y_pred)

y_preds_vmd2 = []
for i in range(K_vmd):
    models_vmd2[i].eval()
    y_pred = []
    with torch.no_grad():
        for inputs, _ in test_loaders_vmd2[i]:
            outputs = models_vmd2[i](inputs)
            y_pred.extend(outputs.cpu().numpy())
    y_pred = np.array(y_pred)
    y_preds_vmd2.append(y_pred)

# 反归一化
y_preds_denormalized_ceemdan = [scalers_ceemdan[i].inverse_transform(y_preds_ceemdan[i]) for i in range(K_ceemdan)]
y_tests_denormalized_ceemdan = [scalers_ceemdan[i].inverse_transform(y_tests_ceemdan[i].cpu().numpy()) for i in range(K_ceemdan)]

y_preds_denormalized_vmd1 = [scalers_vmd1[i].inverse_transform(y_preds_vmd1[i]) for i in range(K_vmd)]
y_tests_denormalized_vmd1 = [scalers_vmd1[i].inverse_transform(y_tests_vmd1[i].cpu().numpy()) for i in range(K_vmd)]

y_preds_denormalized_vmd2 = [scalers_vmd2[i].inverse_transform(y_preds_vmd2[i]) for i in range(K_vmd)]
y_tests_denormalized_vmd2 = [scalers_vmd2[i].inverse_transform(y_tests_vmd2[i].cpu().numpy()) for i in range(K_vmd)]

# 重构预测结果
final_prediction_ceemdan = np.sum(y_preds_denormalized_ceemdan[2:], axis=0)  # 排除CCEMDAN的前两个模态
final_true_ceemdan = np.sum(y_tests_denormalized_ceemdan[2:], axis=0)  # 排除CCEMDAN的前两个模态

final_prediction_vmd1 = np.sum(y_preds_denormalized_vmd1, axis=0)
final_true_vmd1 = np.sum(y_tests_denormalized_vmd1, axis=0)

final_prediction_vmd2 = np.sum(y_preds_denormalized_vmd2, axis=0)
final_true_vmd2 = np.sum(y_tests_denormalized_vmd2, axis=0)

# 合并CCEMDAN和VMD的预测结果
final_prediction = final_prediction_ceemdan + final_prediction_vmd1 + final_prediction_vmd2
final_true = final_true_ceemdan + final_true_vmd1 + final_true_vmd2

# 保存预测结果
predicted_column = 'CCEMDAN-SSA-VMD-WOA-CNN-Resnet-BiGRU-MultiHeadAttention'
os.makedirs('D:/Jupyter/A小论文/Result', exist_ok=True)
predictions_path = os.path.join('D:/Jupyter/A小论文/Result/预测值', 'predictions.csv')

if os.path.exists(predictions_path):
    df = pd.read_csv(predictions_path)
else:
    df = pd.DataFrame()

df.loc[:, predicted_column] = final_prediction.flatten()
df.to_csv(predictions_path, index=False)

# 计算评价指标
r2 = r2_score(final_true, final_prediction)
mape = mean_absolute_percentage_error(final_true, final_prediction)
mse = mean_squared_error(final_true, final_prediction)
rmse = np.sqrt(mse)

print(f'R^2: {r2:.4f}')
print(f'MAPE: {mape:.4f}')
print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')

# 保存模型评价指标
evaluation_path = os.path.join('D:/Jupyter/A小论文/Result/模型评价指标', 'model_evaluation.csv')

if os.path.exists(evaluation_path):
    df_eval = pd.read_csv(evaluation_path)
else:
    df_eval = pd.DataFrame(columns=['Model', 'R^2', 'MAPE', 'MSE', 'RMSE'])

if 'CCEMDAN-SSA-VMD-WOA-CNN-Resnet-BiGRU-MultiHeadAttention' in df_eval['Model'].values:
    df_eval.loc[df_eval['Model'] == 'CCEMDAN-SSA-VMD-WOA-CNN-Resnet-BiGRU-MultiHeadAttention', ['R^2', 'MAPE', 'MSE', 'RMSE']] = [r2, mape, mse, rmse]
else:
    new_row = pd.DataFrame({'Model': ['CCEMDAN-SSA-VMD-WOA-CNN-Resnet-BiGRU-MultiHeadAttention'], 'R^2': [r2], 'MAPE': [mape], 'MSE': [mse], 'RMSE': [rmse]})
    df_eval = pd.concat([df_eval, new_row], ignore_index=True)

df_eval.to_csv(evaluation_path, index=False)

# 预测未来12个月的数据
def predict_next_12_months(model, input_data):
    input_sequence = input_data.clone()
    
    # 预测未来12个月的数据
    future_predictions = []
    for _ in range(12):
        predictions = model(input_sequence[-1].unsqueeze(0))  # 假设 model 是一个 PyTorch 模型
        next_data = torch.cat((input_sequence[-1, 1:], predictions[0].unsqueeze(0)), dim=0)
        input_sequence = torch.cat((input_sequence, next_data.unsqueeze(0)), dim=0)
        future_predictions.append(predictions.squeeze().cpu().detach().numpy())
    future_predictions = np.array(future_predictions)

    return future_predictions

# 假设 X_tests 是一个 PyTorch 张量
future_predictions_ceemdan = [predict_next_12_months(models_ceemdan[i], X_tests_ceemdan[i][-1:]) for i in range(2, K_ceemdan)]  # 排除CCEMDAN的前两个模态
future_predictions_vmd1 = [predict_next_12_months(models_vmd1[i], X_tests_vmd1[i][-1:]) for i in range(K_vmd)]
future_predictions_vmd2 = [predict_next_12_months(models_vmd2[i], X_tests_vmd2[i][-1:]) for i in range(K_vmd)]

# 反归一化预测结果
future_predictions_denormalized_ceemdan = [scalers_ceemdan[i+2].inverse_transform(future_predictions_ceemdan[i].reshape(-1, 1)) for i in range(K_ceemdan-2)]  # 排除CCEMDAN的前两个模态
future_predictions_denormalized_vmd1 = [scalers_vmd1[i].inverse_transform(future_predictions_vmd1[i].reshape(-1, 1)) for i in range(K_vmd)]
future_predictions_denormalized_vmd2 = [scalers_vmd2[i].inverse_transform(future_predictions_vmd2[i].reshape(-1, 1)) for i in range(K_vmd)]

# 重构预测结果
final_future_prediction_ceemdan = np.sum(future_predictions_denormalized_ceemdan, axis=0)
final_future_prediction_vmd1 = np.sum(future_predictions_denormalized_vmd1, axis=0)
final_future_prediction_vmd2 = np.sum(future_predictions_denormalized_vmd2, axis=0)

# 合并CCEMDAN和VMD的未来预测结果
final_future_prediction = final_future_prediction_ceemdan + final_future_prediction_vmd1 + final_future_prediction_vmd2

# 保存预测结果
predicted_column = 'CCEMDAN-SSA-VMD-WOA-CNN-Resnet-BiGRU-MultiHeadAttention'
os.makedirs('D:/Jupyter/A小论文/Result', exist_ok=True)
predictions_path = os.path.join('D:/Jupyter/A小论文/Result/预测值', 'future_predictions.csv')

if os.path.exists(predictions_path):
    df = pd.read_csv(predictions_path)
else:
    df = pd.DataFrame()

df.loc[:, predicted_column] = final_future_prediction.flatten()
df.to_csv(predictions_path, index=False)

# 可视化结果
plt.figure(figsize=(12, 6))

# 绘制测试集的真实数据和预测数据
plt.plot(final_true, label='True (Test Set)', marker='o')
plt.plot(final_prediction, label='Predicted (Test Set)', marker='x')

# 绘制预测的未来12个月
plt.plot(range(len(final_true), len(final_true) + 12), final_future_prediction, label='Predicted Future 12 Months', marker='x')

plt.title('True vs Predicted Data')
plt.xlabel('Time Steps')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

# 可视化损失函数随着训练次数的变化
plt.figure(figsize=(12, 6))
for i in range(2, K_ceemdan):  # 排除CCEMDAN的前两个模态
    plt.plot(train_losses_ceemdan[i], label=f'Train Loss CCEMDAN IMF {i+1}')
    plt.plot(val_losses_ceemdan[i], label=f'Validation Loss CCEMDAN IMF {i+1}')
for i in range(K_vmd):
    plt.plot(train_losses_vmd1[i], label=f'Train Loss VMD1 IMF {i+1}')
    plt.plot(val_losses_vmd1[i], label=f'Validation Loss VMD1 IMF {i+1}')
for i in range(K_vmd):
    plt.plot(train_losses_vmd2[i], label=f'Train Loss VMD2 IMF {i+1}')
    plt.plot(val_losses_vmd2[i], label=f'Validation Loss VMD2 IMF {i+1}')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()